In [2]:

import dask.bag as db
import json
import csv
import requests


In [3]:

# Read the JSONL file (Please provide the correct path to your JSONL file)
# b = db.read_text('../stream_tweetids/streamV2_tweetnet_2023-06.jsons/streamV2_tweetnet_2023-06.jsons').map(json.loads)
b = db.read_text('../stream_tweetids/streamV2_tweetnet_2023-06.jsons/streamV2_tweetnet_2023-06.jsons', blocksize='64MB').map(json.loads)


In [4]:
b.npartitions

18

In [3]:

# Define the transformation function
def extract_fields(json_obj):
    tweet_id = json_obj.get('tweet_id', '')
    tweet_type = json_obj.get('tweet_type', '')
    hashtags = json_obj.get('hashtags', [])
    mentions = json_obj.get('mentions', [])
    return {
        'tweet_id': tweet_id,
        'tweet_type': tweet_type,
        'hashtags': hashtags,
        'mentions': mentions
    }


In [4]:

# API call
def fetch_additional_info(tweet_id):
    url = "https://cdn.syndication.twimg.com/tweet-result"
    querystring = {"id": tweet_id, "lang": "en", "token": "x"}
    payload = ""
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Origin": "https://platform.twitter.com",
    "Connection": "keep-alive",
    "Referer": "https://platform.twitter.com/",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "cross-site",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
    "TE": "trailers"
    }
    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)
    return response.text


In [5]:

# Initialize a CSV writer and write the header
with open('output.csv', 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=['tweet_id', 'tweet_type', 'hashtags', 'mentions', 'lang', 'favorite_count', 'created_at', 'text', 'parent_tweet_id'])
    writer.writeheader()


In [6]:

# Debug Comment: Print the number of partitions in the bag before repartitioning
print('Number of partitions:', b.npartitions)

# Create partitions (for example, 1000 tweets per partition)
n_partition = 100

# Debug Comment: Ensure that n_partition is less than b.npartitions
if n_partition >= b.npartitions:
    print(f'n_partition ({n_partition}) should be less than b.npartitions ({b.npartitions})')


Number of partitions: 1
n_partition (100) should be less than b.npartitions (1)


In [9]:


bag = b.repartition(npartitions=b.npartitions // n_partition)


ZeroDivisionError: division by zero